In [2]:
#ler train.csv
import pandas as pd
df = pd.read_csv('train.csv')

In [5]:
#verificar se há valores nulos e listar apenas as colunas que possuem valores nulos
df.isnull().sum()[df.isnull().sum() > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [24]:
#numero de linhas e colunas de X_cols
X_cols.shape

(37,)

In [25]:
df.shape

(1460, 81)

In [31]:
#df unique
df_not_cols = df.drop(columns = X_cols)

In [32]:
df_not_cols.shape

(1460, 44)

In [41]:
df_not_cols.nunique()

MSZoning           5
Street             2
Alley              2
LotShape           4
LandContour        4
Utilities          2
LotConfig          5
LandSlope          3
Neighborhood      25
Condition1         9
Condition2         8
BldgType           5
HouseStyle         8
RoofStyle          6
RoofMatl           8
Exterior1st       15
Exterior2nd       16
MasVnrType         4
ExterQual          4
ExterCond          5
Foundation         6
BsmtQual           4
BsmtCond           4
BsmtExposure       4
BsmtFinType1       6
BsmtFinType2       6
Heating            6
HeatingQC          5
CentralAir         2
Electrical         5
KitchenQual        4
Functional         7
FireplaceQu        5
GarageType         6
GarageFinish       3
GarageQual         5
GarageCond         5
PavedDrive         3
PoolQC             3
Fence              4
MiscFeature        4
SaleType           9
SaleCondition      6
SalePrice        663
dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [34]:
df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [12]:
#teste train split pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

#separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('SalePrice', axis=1), df['SalePrice'], test_size=0.3, random_state=0)

#Criar X_Cols com apenas colunas numericas
X_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

X_train = X_train[X_cols]
X_test = X_test[X_cols]

#trocar NaN por 0
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [20]:
#criar pipeline Random Forest
from sklearn.ensemble import RandomForestRegressor

pipeline = Pipeline([('scaler', StandardScaler()), ('', RandomForestRegressor())])

#treinar modelo
pipeline.fit(X_train, y_train)

#fazer previsões
y_pred = pipeline.predict(X_test)

#avaliar modelo]
from sklearn.metrics import mean_squared_error
import numpy as np
print(np.sqrt(mean_squared_error(y_test, y_pred)))
#print outras metricas


31593.12574652937


In [49]:
#log do erro
print(np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred))))

0.139431397564966


In [18]:
#describe saleprice
df['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [44]:
#df head Exterior1st
df['Exterior1st'].head(20)

0     VinylSd
1     MetalSd
2     VinylSd
3     Wd Sdng
4     VinylSd
5     VinylSd
6     VinylSd
7     HdBoard
8     BrkFace
9     MetalSd
10    HdBoard
11    WdShing
12    HdBoard
13    VinylSd
14    MetalSd
15    Wd Sdng
16    Wd Sdng
17    MetalSd
18    VinylSd
19    BrkFace
Name: Exterior1st, dtype: object

In [45]:
#read test.csv
df_test = pd.read_csv('test.csv')

In [48]:
#fazer previsao para test.csv, e escrever a resposta no arquivo csv
df_test.fillna(0, inplace=True)
y_pred_test = pipeline.predict(df_test[X_cols])
#fill nan with 0

df_test['SalePrice'] = y_pred_test

df_test[['Id', 'SalePrice']].to_csv('submission.csv', index=False)

